In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import sys
sys.version
#Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from random import shuffle

from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

num_periods_output = 2 #to predict
num_periods_input=4 #input

No_Of_weeks = 4
#Total_No_Features = 15
Total_No_Features = 2

ALL_Test_Data=[]
ALL_Test_Prediction=[]

# Preprocessing

In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

def preprocessing(df,num_features):
    
    df = df.drop(['Occupancy'], axis = 1)
    df['Occupancy'] = df['Occ_percent']
    df = df.drop(['Occ_percent'], axis = 1)

    df['LastUpdated'] = pd.to_datetime(df['LastUpdated'])
    df['Month'] = df['LastUpdated'].dt.month
    df['Day'] = df['LastUpdated'].dt.day
    df['Hour'] = df['LastUpdated'].dt.hour
    df['Minute'] = df['LastUpdated'].dt.minute
     
    # select features
    #with features
   # df=df[['ID','Occupancy','temperature','Month','Day','Hour','Minute','dew_point', 'humidity', 'wind_speed', 'feels_like','Events_Football_City', 'Events_Football_Derby',
    #   'Events_Football_Aston', 'Events_Rugby']]    
    
    #Without features
    df=df[['ID','Occupancy']]

    ################################################encoding########################
    df['Occupancy'] = pd.to_numeric(df['Occupancy'],errors='coerce')
    df['Occupancy'] = df['Occupancy'].abs()
    
        
    Number_Of_Features=num_features
    df=df.values
    df = df.astype('float32')
    split=num_periods_output+num_periods_input  
    
    
    ##################################SPLIT##############################################
    
     ########################## SPLITTING FOR TESTING ##########################
    test_len=np.floor(len(df)*0.2)
    mod=test_len%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    test_len=int(test_len-mod)
    Test=df[(len(df)-test_len):,:]

    ########################### SPLITTING FOR TRAIN ###########################
    
    new_cutted_df=df[:(len(df)-test_len),:]
    Start_train_index=int(np.floor(2*24*7*No_Of_weeks))
    print('instances',Start_train_index)
    Train=new_cutted_df[len(new_cutted_df)-Start_train_index:,:]
    train_len=len(Train)
    mod=train_len%(num_periods_input+num_periods_output)
    #let thelength be divisable by 12
    train_len=int(train_len-mod)
    Train=Train[0:train_len,:]
    print('len Train',len(Train))
   
    '''
    #############################  Normalization on train and validation separatly  #############
     #statistical Features: 'DayOfWeek_Bin_avg','Hour_Bin_avg','DayOfWeek_Hour_Bin_avg'
    ID_Train=Train[:,0]
    Train=np.delete(Train,[0],1)
    #x_batches=x_batches.drop(columns=['ID'], axis=1)
    occ_Train=Train[:,0]
    Train=np.delete(Train,[0],1)
       
    #normalizing data
    Train = Train.astype('float32')
    normalizer = MinMaxScaler().fit(Train)
    #normalizer = Normalizer().fit(Train)
    Train=normalizer.transform(Train)
    
    ID_Train=np.reshape(ID_Train,(len(ID_Train),1))
    occ_Train=np.reshape(occ_Train,(len(occ_Train),1))

    Train=np.append(occ_Train, Train, axis=1)
    Train=np.append(ID_Train, Train, axis=1)'''

    ############################################ TRAIN minibatches ##################################
    
    end=len(Train)
    start=0
    next=0
    x_batches=[]
    y_batches=[]
    
    count=0
    #print('lennnn',len(Train))
    while next+(num_periods_input+num_periods_output)<end:
        next=start+num_periods_input
        x_batches.append(Train[start:next,:])
        y_batches.append(Train[next:next+num_periods_output,1])
        start=start+1
    y_batches=np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)   
    x_batches=np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)   
    print('len x_batches ',len(x_batches))
    
    
    ###########################################TEST#####################################
    '''
    ID_Test=Test[:,0]
    Test=np.delete(Test,[0],1)
    #X_test=X_test.drop(columns=['ID'], axis=1)
    occ_Test=Test[:,0]
    Test=np.delete(Test,[0],1)
        
    Test = Test.astype('float32')
    Test=normalizer.transform(Test)
    
    #------------------
    ID_Test=np.reshape(ID_Test,(len(ID_Test),1))
    occ_Test=np.reshape(occ_Test,(len(occ_Test),1))

    Test=np.append(occ_Test,Test, axis=1)
    Test=np.append(ID_Test, Test, axis=1)
    '''
    ############################################ TEST minibatches ##################################
    end_test=len(Test)
    start_test=0
    next_test=0
    x_testbatches=[]
    y_testbatches=[]
    
    
    #print('lennnn',len(Train))
    while next_test+(num_periods_input+num_periods_output)<end_test:
        next_test=start_test+num_periods_input
        x_testbatches.append(Test[start_test:next_test,:])
        y_testbatches.append(Test[next_test:next_test+num_periods_output,1])
        start_test=start_test+1
    y_testbatches=np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)   
    x_testbatches=np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features) 
    print('len Test',len(Test))
    print('len xTestbatches',len(x_testbatches))
    ######################## Sampling##########################################
    
    #x_batches, y_batches, x_validbatches, y_validbatches, x_testbatches, y_testbatches
    
    return x_batches, y_batches, x_testbatches, y_testbatches

In [0]:
def load_locationfiles(path,loc_id):
    filename=path + '/Birm'+str(loc_id)+'.csv'
    print(filename)
    data_loc=pd.read_csv(filename)
    return data_loc

In [0]:
    data_path=r'/content/drive/My Drive/Birm_Data_Events'

    data_All=pd.DataFrame()
    x_batches_Full=[]
    y_batches_Full=[]
    X_Test_Full=[]
    Y_Test_Full=[]
    range_list = [x for x in range(1,31) if x != 8 and x != 21]

    for loc_id in range_list:
        #========
        data=load_locationfiles(data_path,loc_id)
        header=list(data.columns.values)
        data=pd.DataFrame(data,columns=header)
        x_batches, y_batches,X_Test,Y_Test=preprocessing(data,Total_No_Features)
        #===============================
        for element1 in (x_batches):
            x_batches_Full.append(element1)
            
        for element2 in (y_batches):
            y_batches_Full.append(element2)
                        
        for element5 in (X_Test):
            X_Test_Full.append(element5)
            
        for element6 in (Y_Test):
            Y_Test_Full.append(element6)
    #---------------------shuffle minibatches X and Y together-------------------------------------
    print(len(x_batches_Full),'     length of all file : ',len(y_batches_Full))
    combined = list(zip(x_batches_Full, y_batches_Full))
    random.shuffle(combined)
    shuffled_batch_features, shuffled_batch_y = zip(*combined)  

/content/drive/My Drive/Birm_Data_Events/Birm1.csv
instances 1344
len Train 288
len x_batches  279
len Test 258
len xTestbatches 249
/content/drive/My Drive/Birm_Data_Events/Birm2.csv
instances 1344
len Train 318
len x_batches  309
len Test 252
len xTestbatches 243
/content/drive/My Drive/Birm_Data_Events/Birm3.csv
instances 1344
len Train 306
len x_batches  297
len Test 258
len xTestbatches 249
/content/drive/My Drive/Birm_Data_Events/Birm4.csv
instances 1344
len Train 288
len x_batches  279
len Test 258
len xTestbatches 249
/content/drive/My Drive/Birm_Data_Events/Birm5.csv
instances 1344
len Train 390
len x_batches  381
len Test 234
len xTestbatches 225
/content/drive/My Drive/Birm_Data_Events/Birm6.csv
instances 1344
len Train 390
len x_batches  381
len Test 234
len xTestbatches 225
/content/drive/My Drive/Birm_Data_Events/Birm7.csv
instances 1344
len Train 390
len x_batches  381
len Test 234
len xTestbatches 225
/content/drive/My Drive/Birm_Data_Events/Birm9.csv
instances 1344
len

In [0]:
import itertools
import numpy as np
#xgboost part
print(len(x_batches_Full))
All_Training_Instances=[]

'''Static_Features=[[1,0,0,0,0,0,0,0,1,3,4,6,7,19,9,17,51.3787,-2.3622],[0,1,0,0,0,0,0,0,0,3,9,11,3,29,10,13,51.3843,-2.3686],
                     [0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,51.4113,-2.3869],[0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,51.3902,-2.4059]
                    ,[0,0,0,0,1,0,0,0,0,0,0,4,2,3,0,0,51.3529,-2.3838],[0,0,0,0,0,1,0,0,2,4,8,6,3,39,13,14,51.3842,-2.3590],
                     [0,0,0,0,0,0,1,0,1,1,3,6,6,17,9,7,51.3782,-2.3589],[0,0,0,0,0,0,0,1,1,2,3,6,6,18,9,7,51.3783,-2.3593]]'''
 
#=============== change each window into Instance =================================
for i in range(0,len(shuffled_batch_features)):
    hold=[]

    for j in range(0,len(shuffled_batch_features[i])):
      hold=np.concatenate((hold, shuffled_batch_features[i][j][1]), axis=None) 
    All_Training_Instances.append(hold) 

print(len(All_Training_Instances[0]))

#=================Testing=====================
All_Testing_Instances=[]

#=============== change each window into Instance =================================
print(len(X_Test_Full))
for i in range(0,len(X_Test_Full)):
  hold=[]
  for j in range(0,len(X_Test_Full[i])):
    
    hold=np.concatenate((hold, X_Test_Full[i][j][1]), axis=None)
   
  All_Testing_Instances.append(hold)

print(len(All_Testing_Instances[0]))

8856
4
6750
4


In [0]:
All_Testing_Instances=np.reshape(All_Testing_Instances, (len(All_Testing_Instances),len(All_Testing_Instances[0])))
Y_Test_Full=np.reshape(Y_Test_Full, (len(Y_Test_Full),num_periods_output))

#========== reshape train ==============================
All_Training_Instances=np.reshape(All_Training_Instances, (len(All_Training_Instances),len(All_Training_Instances[0])))
shuffled_batch_y=np.reshape(shuffled_batch_y, (len(shuffled_batch_y),num_periods_output))

print(All_Training_Instances.shape)
model=xgb.XGBRegressor(learning_rate =0.01,
 n_estimators=800,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 seed=27,silent=False)

multioutput=MultiOutputRegressor(model).fit(All_Training_Instances,shuffled_batch_y)

print('Fitting Done!')

prediction=multioutput.predict(All_Testing_Instances)
MSE=np.mean(( prediction- Y_Test_Full)**2)

print('RMSE: ',MSE**0.5)

(8856, 4)
[21:29:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


[21:29:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting Done!
RMSE:  8.560023040027449
